In [2]:
import numpy as np
import pandas as pd

###############################################################################
# 1) READ CSV (NO HEADER), NAME COLUMNS, CREATE DELTA, THEN EXCLUDE delta=0
###############################################################################

# data.csv => L, R, W, Z 
df_data = pd.read_csv("data.csv")

# MI.csv => l, r, w, z, p 
df_MI = pd.read_csv("MI.csv")

# Define delta:
#   delta=0 if R>=1000  => never enters state 2
#   delta=1 if Z<1000   => fully observed 2->3
#   delta=2 otherwise   => enters 2, but 2->3 is censored at Z>=1000
def get_delta(row):
    L, R, W, Z = row["L"], row["R"], row["W"], row["Z"]
    if R >= 1000:
        return 0
    elif Z < 1000:
        return 1
    else:
        return 2

df_data["delta"] = df_data.apply(get_delta, axis=1)

# EXCLUDE all rows with delta=0
df_data = df_data[df_data["delta"] != 0].copy()

###############################################################################
# 2) SPLIT MI RECTANGLES: PROPER (z<1000) 
###############################################################################

df_MI["is_proper"] = df_MI["z"] < 1000

df_proper = df_MI[df_MI["is_proper"]].copy()

###############################################################################
# 3) CONTAINMENT CHECK WITH >= (NOT >)
###############################################################################
def rect_contained_in_obs(l_j, r_j, w_j, z_j,
                          L_i, R_i, W_i, Z_i):
    """
    Returns True if (l_j, r_j] x (w_j, z_j] is contained in
    (L_i, R_i] x (W_i, Z_i].
    Using non-strict inequalities on the left side: l_j >= L_i, w_j >= W_i.
    """
    # For a valid containment:
    #   L_i <= l_j < r_j <= R_i
    #   W_i <= w_j < z_j <= Z_i
    cond12 = (l_j >= L_i) and (r_j <= R_i)
    cond23 = (w_j >= W_i) and (z_j <= Z_i)
    return (cond12 and cond23)


###############################################################################
# 4) MULTIPLE IMPUTATION FOR DELTA=1 OR 2 ONLY
###############################################################################

def impute_one_obs(obs_row, df_proper):
    """
    obs_row: a row from df_data, which now must have delta in {1,2} only.
    - If delta=1 => pick from df_proper, reduce to (r_j, z_j).
    """

    L_i, R_i, W_i, Z_i = obs_row["L"], obs_row["R"], obs_row["W"], obs_row["Z"]
    delta_i = obs_row["delta"]

    if delta_i == 1:
        # Proper
        cand_mask = df_proper.apply(
            lambda r: rect_contained_in_obs(r["l"], r["r"], r["w"], r["z"],
                                            L_i, R_i, W_i, Z_i),
            axis=1
        )
        df_cand = df_proper[cand_mask]
        chosen = df_cand.iloc[0]

        return chosen["r"], chosen["z"], 1

    else:
        raise ValueError("impute_one_obs called with invalid delta (not 1 or 2).")


###############################################################################
# 5) COMPUTE THE TITMAN–PUTTER TEST STATISTIC FOR A GIVEN s
###############################################################################

def compute_test_stat(reduced_obs, s_val):
    """
    U_s^{(1)}(2,3) = sum over distinct 2->3 times t_v of:
        [ sum_i (delta1_i(s)* dN_i(t_v))
          - ( (sum_i delta1_i(s)*Y_i(t_v)) * (sum_i dN_i(t_v)) / (sum_i Y_i(t_v)) ) ]
    """
    # Distinct 2->3 times among delta=1 => second_i
    event_times = []
    for (r_i, second_i, d_i) in reduced_obs:
        if d_i == 1:
            event_times.append(second_i)
    event_times = np.unique(event_times)
    if len(event_times) == 0:
        return 0.0

    def in_state1_at_s(r_i, s_val):
        # Subject is in state1 at time s if r_i >= s
        return (r_i >= s_val)

    def dN_i(r_i, sec_i, d_i, t_v):
        # 2->3 transition at t_v if delta=1 and sec_i == t_v
        return 1 if (d_i==1 and sec_i == t_v) else 0

    def Y_i(r_i, sec_i, d_i, t_v):
        # subject i enters state2 at r_i < t_v
        # If delta=1 => 2->3 time is sec_i
        # If delta=2 => censor at sec_i. Must not have left state2 by t_v
        if r_i >= t_v:
            return 0
        if d_i == 1:
            return 1 if (sec_i >= t_v) else 0
        elif d_i == 2:
            return 1 if (sec_i > t_v) else 0
        else:
            return 0

    U_s = 0.0
    for t_v in event_times:
        sum_delta1_dN = 0
        sum_dN = 0
        sum_Y = 0
        sum_delta1_Y = 0

        for (r_i, sec_i, d_i) in reduced_obs:
            di_s = 1 if in_state1_at_s(r_i, s_val) else 0
            dn_i = dN_i(r_i, sec_i, d_i, t_v)
            y_i = Y_i(r_i, sec_i, d_i, t_v)

            sum_delta1_dN += di_s * dn_i
            sum_dN += dn_i
            sum_Y += y_i
            sum_delta1_Y += di_s * y_i

        if sum_Y > 0:
            correction = (sum_delta1_Y * sum_dN) / float(sum_Y)
        else:
            correction = 0.0

        U_s += (sum_delta1_dN - correction)

    return U_s

###############################################################################
# 6) MAIN FUNCTION: MULTIPLE IMPUTATIONS => TEST STAT
###############################################################################

def run_markov_test(df_data, df_proper,
                    s_values=range(8,21),
                    K=500):
    """
    For each s in s_values:
      - Do K imputations
      - Collect the K test stats T_k
      - Output mean(T_k), approximate SE, and Z_approx = mean / (SE_of_mean)
    """
    df_data = df_data[df_data["delta"] == 1]
    print(df_data)

    results = []
    for s_val in s_values:
        T_vals = []
        for _ in range(K):
            # Impute every subject
            reduced_list = []
            for _, row_i in df_data.iterrows():
                r_star, sec_star, d_star = impute_one_obs(row_i, df_proper)
                reduced_list.append((r_star, sec_star, d_star))

            # Compute test statistic
            T_val = compute_test_stat(reduced_list, s_val)
            T_vals.append(T_val)

        # Combine across K imputations
        arr_T = np.array(T_vals)
        mean_T = arr_T.mean()
        var_T = arr_T.var(ddof=1)   # sample variance of T_k
        # naive standard error of the *mean* is sqrt(var_T/K)
        se_mean = np.sqrt(var_T / K) if K>1 else 0.0
        z_approx = mean_T / se_mean if se_mean>0 else np.nan

        results.append((s_val, mean_T, se_mean, z_approx))

    df_res = pd.DataFrame(results, columns=["s","mean_test_stat","SE_mean","Z_approx"])
    return df_res


###############################################################################
# 7) EXECUTE THE TEST
###############################################################################

if __name__ == "__main__":
    # E.g. K=500 imputations
    df_results = run_markov_test(df_data, df_proper, range(8,9), K=100)
    print(df_results)

      L   R   W   Z  delta
28    1  10  11  12      1
29    1  15  20  21      1
30    5   8  13  13      1
31    9  13  14  15      1
32    9  13  17  18      1
33   10  11  15  16      1
34   10  12  16  17      1
35   13  14  17  18      1
36   13  14  20  21      1
61    1   7  12  13      1
62    3   7  17  17      1
63    7   9  19  19      1
64    9  11  17  18      1
65    9  12  22  22      1
66   10  12  15  16      1
67   10  12  23  23      1
68   13  15  17  18      1
69   13  15  17  18      1
70   14  15  23  23      1
96    1   7  16  16      1
97    5   7  11  12      1
98    8  10  15  15      1
99    9  12  17  18      1
100  10  11  14  15      1
101  10  14  15  16      1
102  12  13  20  20      1
103  13  14  19  20      1
   s  mean_test_stat       SE_mean      Z_approx
0  8       -3.981684  8.926529e-17 -4.460507e+16
